In [1]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2


2023-11-24 14:54:33.416391: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
interpreter = tf.lite.Interpreter(model_path="../models/3.tflite")
interpreter.allocate_tensors()


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [3]:
def draw_key_points(frame, keypoints, conf_threshold):
    y, x, c = frame.shape
    shaped = np.squeeze(np.multiply(keypoints, [y,x,1]))
    i = 0
    for kp in shaped:
        ky, kx, kp_conf = kp
        if kp_conf > conf_threshold:
            cv2.circle(frame,(int(kx), int(ky)), 1, (0, 255, 0), 5)
            cv2.circle(frame,(int(10), int(10)), 2, (0, 255, 0), 5)
        if i == 0:
            print(ky, ky/720, kx, kx/1280)
        i += 1
    return frame


In [28]:
frame.shape


(720, 1280, 3)

In [5]:
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()

    img = frame.copy()
    img = cv2.resize(img, (192, 192))
    img = tf.image.resize_with_pad(np.expand_dims(img, axis=0), 192, 192)
    input_image = tf.cast(img, dtype=tf.float32)

    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    interpreter.set_tensor(input_details[0]["index"], np.array(input_image))
    interpreter.invoke()
    key_points_with_scores = interpreter.get_tensor(output_details[0]["index"])

    draw_key_points(frame,key_points_with_scores, conf_threshold=0.5)

    cv2.imshow("Live Feed Test", frame)

    if cv2.waitKey(10) & 0xFF==ord("q"):
        break

cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)


429.2827033996582 0.5962259769439697 699.8175811767578 0.546732485294342
449.37729835510254 0.6241351366043091 701.4178466796875 0.5479826927185059
461.05589389801025 0.6403554081916809 698.0938720703125 0.5453858375549316
456.2970542907715 0.6337459087371826 699.3311309814453 0.5463524460792542
453.96374702453613 0.6305052042007446 702.1372222900391 0.548544704914093
451.95213317871094 0.627711296081543 698.9210510253906 0.5460320711135864
452.8412103652954 0.6289461255073547 698.5138702392578 0.5457139611244202
452.8110408782959 0.6289042234420776 700.5178070068359 0.5472795367240906
455.5260372161865 0.632675051689148 705.4603576660156 0.5511409044265747
455.00053882598877 0.6319451928138733 707.3399353027344 0.5526093244552612
454.0946388244629 0.6306869983673096 712.6421356201172 0.5567516684532166
449.5481014251709 0.6243723630905151 705.6999206542969 0.5513280630111694
449.6031618118286 0.624448835849762 712.2773742675781 0.5564666986465454
448.2926559448242 0.6226286888122559 7

-1